# Enter State Farm

In [1]:
%matplotlib inline
import imp
import utils; imp.reload(utils)
from utils import *
from __future__ import division, print_function
path = "/home/carnd/hack/data/state/sample/"
from IPython.display import FileLink

Using TensorFlow backend.


In [2]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [5]:
%cd data/state

/home/carnd/hack/data/state


In [6]:
%cd train

/home/carnd/hack/data/state/train


In [7]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [8]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [9]:
from shutil import copyfile

In [10]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [11]:
%cd ../valid

/home/carnd/hack/data/state/valid


In [12]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(500): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [13]:
%cd ../../..

/home/carnd/hack


In [14]:
%mkdir data/state/results

In [15]:
%mkdir data/state/sample/test

## Create batches

In [16]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [17]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 1000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [18]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [19]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1000/1000 [==============================] - 35s - loss: 13.0107 - acc: 0.1250 - val_loss: 13.6595 - val_acc: 0.1500
Epoch 2/2
1000/1000 [==============================] - 29s - loss: 13.9486 - acc: 0.1280 - val_loss: 13.6654 - val_acc: 0.1500


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [21]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [22]:
np.round(model.predict_generator(batches, batches.n)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [23]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1000/1000 [==============================] - 35s - loss: 2.4621 - acc: 0.1560 - val_loss: 5.9250 - val_acc: 0.1280
Epoch 2/2
1000/1000 [==============================] - 29s - loss: 1.9380 - acc: 0.3190 - val_loss: 4.0623 - val_acc: 0.1700


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [24]:
model.optimizer.lr=0.001

In [25]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1000/1000 [==============================] - 35s - loss: 1.6097 - acc: 0.4830 - val_loss: 2.7763 - val_acc: 0.2660
Epoch 2/4
1000/1000 [==============================] - 29s - loss: 1.3070 - acc: 0.6340 - val_loss: 2.1578 - val_acc: 0.3300
Epoch 3/4
1000/1000 [==============================] - 29s - loss: 1.1619 - acc: 0.6970 - val_loss: 1.7476 - val_acc: 0.4100
Epoch 4/4
1000/1000 [==============================] - 29s - loss: 1.0021 - acc: 0.7690 - val_loss: 1.3710 - val_acc: 0.5340


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [26]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 500 images belonging to 10 classes.


In [27]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 1.37,  0.53],
       [ 1.35,  0.55],
       [ 1.4 ,  0.52],
       [ 1.39,  0.54],
       [ 1.37,  0.53],
       [ 1.38,  0.53],
       [ 1.39,  0.54],
       [ 1.39,  0.53],
       [ 1.37,  0.53],
       [ 1.36,  0.54]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [28]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1000/1000 [==============================] - 35s - loss: 5.8209 - acc: 0.1850 - val_loss: 12.5997 - val_acc: 0.1140
Epoch 2/2
1000/1000 [==============================] - 30s - loss: 2.4943 - acc: 0.4880 - val_loss: 4.5383 - val_acc: 0.3580


In [29]:
model.optimizer.lr=0.001

In [30]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1000/1000 [==============================] - 35s - loss: 1.0627 - acc: 0.7380 - val_loss: 3.0723 - val_acc: 0.4820
Epoch 2/4
1000/1000 [==============================] - 30s - loss: 0.7182 - acc: 0.8370 - val_loss: 1.6724 - val_acc: 0.6520
Epoch 3/4
1000/1000 [==============================] - 30s - loss: 0.4788 - acc: 0.9140 - val_loss: 1.1433 - val_acc: 0.7460
Epoch 4/4
1000/1000 [==============================] - 30s - loss: 0.3643 - acc: 0.9670 - val_loss: 0.8529 - val_acc: 0.8360


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [31]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

# model.optimizer.lr = 0.01
# model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
#                  nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1000/1000 [==============================] - 37s - loss: 2.2530 - acc: 0.2580 - val_loss: 8.0412 - val_acc: 0.2000
Epoch 2/2
1000/1000 [==============================] - 32s - loss: 1.2528 - acc: 0.6390 - val_loss: 5.1401 - val_acc: 0.2080


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [33]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
#     model.optimizer.lr = 0.001
#     model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
#                      nb_val_samples=val_batches.nb_sample)
#     return model

In [ ]:
conv1(batches)

Epoch 1/2


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Height shift: move the image up and down -

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Random shear angles (max in radians) -

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Rotation: max in degrees -

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Channel shift: randomly changing the R,G,B colors - 

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Lucky we tried that - we starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.